# Hands-On Exercise 8.1: Handle Update Events

## Objective

Update a Hi-Lo game to send event notifications when scores change.

Close the previous solution if you have not already done so.

#### Open the solution `HiLoGame.sln` found in the directory `C:\Course\510D\Exercises\Ex081\`.

The solution contains a simple WinForms version of the **Hi-Lo** number-guessing game.

Launch the application and open **View | Attempts...**. This displays a window that keeps track of the average number of attempts that were required to guess the secret number. It will update on a correct guess.

Play a couple of games to understand how the application operates.

Review the code in `Game.cs`. The `MakeGuess` method is where the majority of the work is performed.

Right-click on `MainForm.cs` in the **Solution Explorer** and select **View Code**.

Look at the `guessButton_Click` handler. When a guess is made, this triggers updates to

- `messageLabel`
- `attemptsLabel`
- `_scoreForm` (on a _successful_ guess)

`MainForm` can do this becauses it's making assumptions about how the `Game` class' logic works. This doesn't feel like a very good separation of concerns. 

What if, for example, `Game` is changed to penalized tardy users with an additional attempt? `MainForm` would then fail to display the correct number of attempts taken.

In addition, `MainForm` is taking responsibility for updating the attempts history window. It's doing too much.

#### Have the `Game` class notify observers when number of attempts change.

Add a `GuessMade` event to the `Game` class. This should look something like

```C#
public event EventHandler? GuessMade;
```

In the `MakeGuess` method, add the following code, just before the `return` statement.

```C#
GuessMade?.Invoke(this, EventArgs.Empty);
```

We need the `?` operator to make sure that there's at least one observer before we send the notifications.

Get a clean build.

#### Listen for `GuessMade` events in `MainForm`.

Open `MainForm` in the code view.

Add an event handler called `UpdateAttempts` that will update `attemptsLabel` when a `GuessMade` notification is received.

The code will look like

```C#
private void UpdateAttempts(object? sender, EventArgs e)
{
    attemptsLabel.Text = $"{_game.CurrentAttempts} attempt(s)";
}
```

Note the parameters of this method. They must match the definition of the event handler.

In `MainForm_Load` register this handler with the event as follows.

```C#
attemptsLabel.Text = $"{_game.CurrentAttempts} attempt(s)";
```

We can remove the following statement from the `guessButton_Click` event handler.
    
```C#
attemptsLabel.Text = $"{_game.CurrentAttempts} attempt(s)";
```

Now, if the `Game` class surprises `MainForm` with a change to the scoring, it'll be handled appropriately.

Run and test the application.

## Congratulations! You have completed the exercise. If you have more time, please continue to the bonus.

# Bonus (Optional)

Add an event to notify listeners of _successful_ guesses and use them to update `ScoreForm`'s history.

#### Create a successful guess event in `Game`.

Add a `SuccessfulGuessMade` event to `Game`. The code will look like

```C#
public event EventHandler? SuccessfulGuessMade;
```

Invoke the event just before the end of the `MakeGuess` method. The code should be similar to the following.
    
```C#
if (guessStatus == GuessStatus.Correct)
{
    SuccessfulGuessMade?.Invoke(this, EventArgs.Empty);
}
```

Get a clean build.

#### Register a `SuccessfulGuessMade` handler in `ScoreForm`.

Create an update handler (replacing the existing `Update` method). It should look like

```C#
private void Update(object? sender, EventArgs e)
{
    scoresListBox.Items.Insert(
        0, 
        $"At {DateTime.Now.ToShortTimeString()} the average # of attempts was {_game!.AverageAttempts:N2}"
    );
}
```

Create a `public Initialize` method that will received a `Game` object (from `MainForm`).

Store it in a `private` field (so the # of attempts can be retrieved later) and register the event handler.

The new code should look like

```C#
private Game? _game;

public void Initialize(Game game)
{
    _game = game;

    game.SuccessfulGuessMade += Update;
}
```

Get a clean build.

#### Initialize `ScoreForm` from `MainForm`.

Open `MainView` in code view.

In `MainForm_Load` pass the `_game` to `_scoreForm` via it's `Initialize` method. The statement should be similar to    

```C#
_scoreForm.Initialize(_game);
```

Run and test the application.

## Congratulations! You have completed the bonus.